## 5. Model Evaluation and Analysis

5.1 Obtain the model's prediction on the test set.

In [ ]:
 model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for img, label in test_loader:
            img, label = img.to(device), label.to(device)

            out = model(img)

            loss = loss_fn(out, label)
            test_loss += loss.item()


            _, predicted = torch.max(out, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()

    test_accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{n_epochs}, Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {test_accuracy:.2f}%")


Epoch 20/20, Test Loss: 1.6853, Test Accuracy: 99.90%


5.2 Calculate the report the following metrics:
- accuracy
- precision
- recall

5.3: Discuss the model's performance.

In [ ]:
#This model has performed exceptionally well. I have had to tinker with the design a lot with diffrent loss functions,learning rate. I realized I did not do the data loader properly which is why my code did not run well.
#finally I was able to achive a 99% accurary which is increadible in this context. I was expecting a 70% accuracy but somehow adding x = self.relu2(x) x = self.fc3(x), these 2 parameters did the job.
#The lost primarily was not stable with numbers over 1000 which was extreme. Later I had to look into the tranning and testing losses symontaniusly which allowed me better understadning and I adjusted accordningly.I am happy with the results now I want to check it out visually.

## 6. Data Augmentation and Retraining

To enhance the model's performance, it is essential to increase the number of malignant instances. Apply random shifting and rotation to generate new training instances, ensuring an equal number of instances in each class within the training set.

6.1 Augment the number of malignent instances in the training set.

In [ ]:

    augmentation_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.RandomAffine(0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
])

  # positive class
    train_chunks_augmented = []
    train_labels_augmented = []

    for img, label in zip(train_chunks, train_labels):
      if label == 1:
        augmented_img = augmentation_transforms(img)
        train_chunks_augmented.append(augmented_img)
        train_labels_augmented.append(label)
    for img, label in zip(train_chunks, train_labels):
      if label == 0:
        train_chunks_augmented.append(img)
        train_labels_augmented.append(label)

  # augmented data into tensors
    train_chunks_augmented = torch.stack(train_chunks_augmented)
    train_labels_augmented = torch.tensor(train_labels_augmented, dtype=torch.long)

    train_dataset = TensorDataset(train_chunks_augmented, train_labels_augmented)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    print(f"Augmented batch shape: {next(iter(train_loader))[0].shape}")


Augmented batch shape: torch.Size([32, 32, 48, 48])


6.2 Retrain the neural network model on the new training set.

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, output_size):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, hidden_size_1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size_2, output_size)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

input_size = 73728
hidden_size_1 = 128
hidden_size_2 = 64
output_size = 2

model = SimpleNN(input_size, hidden_size_1, hidden_size_2, output_size)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)


n_epochs = 20
for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    for img, label in train_loader:
        img, label = img.to(device), label.to(device)


        out = model(img)


        loss = loss_fn(out, label)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()


    print(f"Epoch {epoch+1}/{n_epochs}, Training Loss: {running_loss/len(train_loader):.4f}")


    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for img, label in test_loader:
            img, label = img.to(device), label.to(device)


            out = model(img)


            loss = loss_fn(out, label)
            test_loss += loss.item()


            _, predicted = torch.max(out, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()

    test_accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{n_epochs}, Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {test_accuracy:.2f}%")

Epoch 1/20, Training Loss: 83.8977
Epoch 1/20, Test Loss: 55.8334, Test Accuracy: 99.80%
Epoch 2/20, Training Loss: 47.5911
Epoch 2/20, Test Loss: 6.0229, Test Accuracy: 99.90%
Epoch 3/20, Training Loss: 10.4297
Epoch 3/20, Test Loss: 9.2878, Test Accuracy: 98.50%
Epoch 4/20, Training Loss: 7.9931
Epoch 4/20, Test Loss: 12.8115, Test Accuracy: 99.90%
Epoch 5/20, Training Loss: 10.9177
Epoch 5/20, Test Loss: 16.2670, Test Accuracy: 99.90%
Epoch 6/20, Training Loss: 4.8276
Epoch 6/20, Test Loss: 13.0005, Test Accuracy: 99.90%
Epoch 7/20, Training Loss: 0.2658
Epoch 7/20, Test Loss: 8.0462, Test Accuracy: 99.90%
Epoch 8/20, Training Loss: 0.0884
Epoch 8/20, Test Loss: 8.8153, Test Accuracy: 99.90%
Epoch 9/20, Training Loss: 0.0008
Epoch 9/20, Test Loss: 8.5869, Test Accuracy: 99.90%
Epoch 10/20, Training Loss: 0.0008
Epoch 10/20, Test Loss: 8.5868, Test Accuracy: 99.90%
Epoch 11/20, Training Loss: 0.0008
Epoch 11/20, Test Loss: 8.5868, Test Accuracy: 99.90%
Epoch 12/20, Training Loss: 0.0

6.3 Perform model evaluation and compare the performance of the new model to the old model.

In [ ]:
"""I’m honestly pretty surprised by the results. When I compared the two models, the old one performed way better than I expected. The training loss started at 83.8977 and gradually dropped all the way down to 0.0007,
with the test loss also going down and stabilizing at a very low value. The test accuracy remained a solid 99.90% throughout, which really shows how well it learned and generalized. But the new model, it's a bit of a shocker.
The training loss dropped to 0.0007 quickly, but the test loss just got stuck at 1.6853, even though the test accuracy stayed at 99.90%. It feels like the new model is just underfitting, not improving much despite its accuracy.
I’m really surprised that the old model outperforms it so much in terms of optimization and learning!
""""


